In [8]:
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
from torch.utils.data import DataLoader, Dataset

ImportError: cannot import name 'is_tokenizers_available' from 'transformers.utils' (/usr/local/miniconda3/envs/tsum/lib/python3.8/site-packages/transformers/utils/__init__.py)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_folder = "../pretrained_models"
model_name = "bert-base-uncased"
model_path = os.path.join(model_folder, model_name)
learning_rate = 1e-5
batch_size = 32
num_epochs = 8

In [3]:
def get_data_and_preprocess(texts_path, labels_path):
    texts = pd.read_csv(texts_path, header=0, names=['ID', 'Sentence 0', 'Sentence 1'])
    labels = pd.read_csv(labels_path, header=None, names=['ID', 'Answer'])['Answer']
    for i in range(len(texts)):
        if not texts.iloc[i, 1].endswith("."):
            texts.iloc[i, 1] += "."
        if not texts.iloc[i, 2].endswith("."):
            texts.iloc[i, 2] += "."
    return texts, labels

In [4]:
train_texts, train_labels = get_data_and_preprocess(
    '../data/Training Data/subtaskA_data_all.csv',
    '../data/Training Data/subtaskA_answers_all.csv'
)

val_texts, val_labels = get_data_and_preprocess(
    '../data/Dev Data/subtaskA_dev_data.csv',
    '../data/Dev Data/subtaskA_gold_answers.csv'
)

test_texts, test_labels = get_data_and_preprocess(
    '../data/Test Data/subtaskA_test_data.csv',
    '../data/Test Data/subtaskA_gold_answers.csv'
)

# Use the validation dataset as the training dataset
train_texts = pd.concat([train_texts, val_texts], axis=0, ignore_index=True)
train_labels = pd.concat([train_labels, val_labels], axis=0, ignore_index=True)
train_data = pd.concat([train_texts, train_labels], axis=1)
train_data

ID                                         Sentence 0  \
0         0              He poured orange juice on his cereal.   
1         1                                   He drinks apple.   
2         2                             Jeff ran a mile today.   
3         3                              A mosquito stings me.   
4         4                               A niece is a person.   
...     ...                                                ...   
10992   700                      THE TEMPERATURE IS EMOTIONAL.   
10993   355                    Blended up fruit makes a nacho.   
10994  1550                           Dogs falls from the sky.   
10995  1364                   He put a dinosaur in the fridge.   
10996  1257  The cleaner is in charge of the money at the s...   

                                              Sentence 1  Answer  
0                          He poured milk on his cereal.       0  
1                                        He drinks milk.       0  
2                          Jeff ran 100,000 miles today.       1  
3                                    I sting a mosquito.       1  
4                                 A giraffe is a person.       1  
...                                                  ...     ...  
10992                           THE TEMPERATURE IS WARM.       0  
10993                 Blended up fruit makes a smoothie.       0  
10994                           Rain falls from the sky.       0  
10995                    He put a chicken in the fridge.       0  
10996  The cashier is in charge of the money at the s...       0  

[10997 rows x 4 columns]

In [5]:
# Tokenization for texts
tokenizer = BertTokenizer.from_pretrained(model_path)
train_encodings = tokenizer(list(train_texts["Sentence 0"]), list(train_texts["Sentence 1"]), padding=True)
test_encodings = tokenizer(list(test_texts["Sentence 0"]), list(test_texts["Sentence 1"]), padding=True)

In [6]:
class ComVEDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(value[idx]) for key, value in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
# Turn encodings and labels into a Dataset object
train_dataset = ComVEDataset(train_encodings, train_labels)
test_dataset = ComVEDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [10]:
class BertForSequenceClassification2(torch.nn.Module):
    def __init__(self, bert_model):
        super().__init__()
        self.bert = bert_model
        self.cls = torch.nn.Linear(768, 2)
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_out = self.bert(input_ids, attention_mask, token_type_ids)[0][:, 0]
        cls_out = self.cls(bert_out)
        return cls_out

In [11]:
bert_model = BertModel.from_pretrained(model_path)
model = BertForSequenceClassification2(bert_model).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [12]:
# Start training
model.train()

train_loss = list()
train_accuracies = list()
highest_accuracy = 0
for epoch in range(num_epochs):
    print('Epoch %s/%s' % (epoch + 1, num_epochs))
    
    correct = 0
    count = 0
    epoch_loss = list()
    
    pbar = tqdm(train_loader)
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask, token_type_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # make predictions
        predictions = outputs.argmax(dim=1)

        # count accuracy
        correct += predictions.eq(labels).sum().item()
        count += len(labels)
        accuracy = correct * 1.0 / count

        # show progress along with metrics
        pbar.set_postfix({
            'Loss': '{:.3f}'.format(loss.item()),
            'Accuracy': '{:.3f}'.format(accuracy)
        })
        
        # record the loss for each batch
        epoch_loss.append(loss.item())
        
    pbar.close()
    
    # record the loss and accuracy for each epoch
    train_loss += epoch_loss
    train_accuracies.append(accuracy)


Epoch 1/8


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:07<00:00,  5.11it/s, Loss=0.361, Accuracy=0.610]


Epoch 2/8


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:03<00:00,  5.45it/s, Loss=0.312, Accuracy=0.854]


Epoch 3/8


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:03<00:00,  5.38it/s, Loss=0.011, Accuracy=0.984]


Epoch 7/8


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:04<00:00,  5.35it/s, Loss=0.017, Accuracy=0.990]


Epoch 8/8


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:03<00:00,  5.43it/s, Loss=0.001, Accuracy=0.992]


In [13]:
# Start testing
model.eval()

with torch.no_grad():
    
    correct = 0
    count = 0
    record = list()
    
    pbar = tqdm(test_loader)
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        outputs = model(input_ids, attention_mask, token_type_ids)
        # make predictions
        predictions = outputs.argmax(dim=1)

        # count accuracy
        correct += predictions.eq(labels).sum().item()
        count += len(labels)
        accuracy = correct * 1.0 / count

        # show progress along with metrics
        pbar.set_postfix({
            'loss': '{:.3f}'.format(loss.item()),
            'accuracy': '{:.3f}'.format(accuracy)
        })
    
        # record the results
        record.append((int(labels),int(predictions)))
        
    pbar.close()
    
print("The final accuracy on the test dataset: %s%%" % round(accuracy*100,4))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 71.40it/s, loss=0.001, accuracy=0.877]

The final accuracy on the test dataset: 87.7%
